# Bellman Rank

参考：
* [RL Theory and Algorithms](https://rltheorybook.github.io/rltheorybook_AJKS.pdf)
* [Contextual Decision Processes with Low Bellman Rank are PAC-Learnable](https://arxiv.org/abs/1610.09512)：V-Bellman rankが初めて登場した論文．
* [COLT RL Tutorial 2021](https://rltheorybook.github.io/colt21_part3.pdf)：Bellman rankのモチベーションがわかりやすい

Linear MDPやLinear banditでは線形な遷移や報酬の構造を考え，そしてそれを解くために線形なQ関数を考えました．今回はより一般的なMDPの構造と，それを解くための仮説を考えてみます．

表記：
* 今回はPAC RLを考えます．
* $a_{0: h} \sim d^\pi$：$\pi$における軌跡からサンプルされた行動$a_0$から$a_h$
    * 他もだいたい同様

## 仮説集合

各ホライゾンのステップにおいて何らかの仮説を選択する状況を考えます．つまり，仮説集合：$\mathcal{H}=\mathcal{H}_0 \times \ldots \times \mathcal{H}_{H-1}$を考え，
選択した仮説$f \in \mathcal{H}_h$を使って，そのステップに置ける$Q_{h, f}$や$V_{h, f}$を構築します．

また，この仮説集合は$V_{h, f}(s)=\max _a Q_{h, f}(s, a)$ for all $f \in \mathcal{H}, h \in[H]$, and $s \in \mathcal{S}$であるように構築します．
これはこの仮定をみたさないような仮説を取り除けば常に構築することができます．

$\pi_{h, f}(s)=\operatorname{argmax}_{a \in \mathcal{A}} Q_{h, f}(s, a)$を$Q_{h, f}$に対して貪欲な方策とします．
また，$\left\{\pi_{h, f}\right\}_{h=0}^{H-1}$とします．

このとき，例えば次のような仮説集合が考えられます：

---

**Q値とV値をそのまま表す仮説集合**

$$\mathcal{H}_h \subset\left\{\left(Q_h, V_h\right) \mid Q_h \text{ is a function from } \mathcal{S} \times \mathcal{A} \mapsto \mathbb{R} \text{ and }V_h \text{ is a function from } \mathcal{S} \mapsto \mathbb{R}\right\}$$

このとき，どの仮説$f:=\left(\left(Q_0, V_0\right),\left(Q_1, V_1\right), \ldots,\left(Q_{H-1}, V_{H-1}\right)\right) \in \mathcal{H}$についても，対応するQ値とV値を$Q_{h, f}=Q_f$および$V_{h, f}=V_f$になるように選べます．

---

**Q値だけ表す仮説集合**

$$
\mathcal{H}_h \subset\left\{Q_h \mid Q_h \text { is a function from } \mathcal{S} \times \mathcal{A} \mapsto \mathbb{R}\right\}
$$

このときはV値は$V_{h, f}(\cdot)=\max _{a \in \mathcal{A}} Q_{h, f}(\cdot, a)$として選択します．

---

**モデルベースの仮説集合**

$$
\mathcal{H}_h \subset\left\{\left(P_h, r_h\right) \mid P_h \text { is a function from } \mathcal{S} \times \mathcal{A} \mapsto \Delta(\mathcal{S})\text { and }
r_h \text { is a function from } \mathcal{S} \times \mathcal{A} \mapsto \mathbb{R}\right\}
$$

このときは，どんな仮説$f:=\left(\left(P_0, r_0\right),\left(P_1, r_1\right), \ldots,\left(P_{H-1}, r_{H-1}\right)\right) \in \mathcal{H}$に対しても，そのQ値とV値を対応するMDPの最適価値を求めれば計算できます．

---

**実現可能な仮説集合**

$Q_h^{\star}(s, a)=Q_{h, f^{\star}}(s, a)$を満たすような仮説が$f^\star \in \mathcal{H}$であるとき，$\mathcal{H}$は実現可能といいます．

## Bellman rank

参考
* [Contextual Decision Processes with Low Bellman Rank are PAC-Learnable](https://arxiv.org/abs/1610.09512)：V-Bellman rankが初めて登場した論文．
* [COLT RL Tutorial 2021](https://rltheorybook.github.io/colt21_part3.pdf)：Bellman rankのモチベーションがわかりやすい

---

**モチベーション（COLTをちゃんと見ると良さそう）**

[RL_LowerBound_statistical_limits.ipynb](RL_LowerBound_statistical_limits.ipynb)でやったように，$Q^\pi$の線形実現可能性だけではMDPを多項式時間で解くことはできません．
一方で，Linear MDPのようにMDPが線形な構造をしている場合，線形な仮説集合を使えば，多項式時間で解くことができます．

一般的にはどんな構造と仮説集合を考えれば簡単に解けるでしょうか？
MDPの基本的な解き方として，ベルマン誤差の最小化があります．
そこでは，何らかの仮説で探索して集めたデータに対して，誤差を最小化するように仮説が選択されます（Linear MDPを思い出しましょう）．

そこで，仮説$f$（データを集めるやつ）と仮説$g$（Fittingするやつ）について，次のようなベルマン誤差を定義しましょう：

$$
\mathcal{E}_h(f, g)=
\mathbb{E}_{a_{0: h} \sim d^{\pi_f}}\left[Q_{h, g}\left(s_h, a_h\right)-r\left(s_h, a_h\right)-V_{h+1, g}\left(s_{h+1}\right)\right]
$$

* これはつまり，ロールイン方策$\pi_f$に従って動きまわってステップ$h$に至るとき，ステップ$h$と$h+1$において，仮説$g$によって発生するベルマン誤差の値を表しています．
* 例えば$g=Q^\star$であれば，これはどの$\pi_f$に対しても$\mathcal{E}_h(f, g)=0$を満たします．

このように，仮説集合に対して，データを集める仮説$f$とFittingする仮説$g$の２つに対して，ステップ$h$でのベルマン誤差が定義できます．

このベルマン誤差がどのような構造を持っていれば簡単に解けそうでしょうか？
基本的なアプローチとしてOptimisticな手法を取るならば，仮説集合が十分な速度で小さくなれば良いです．
そこで，$f, g$個目の要素が$\mathcal{E}_h(f, g)$になっているような$|\mathcal{H}| \times |\mathcal{H}|$の行列を考えましょう．

![Bellman-rank](figs/Bellman-rank.png)

この行列のランクが小さい時，何が起こるでしょうか？
例えば$\pi_{f_1}$および$\pi_{f_2}$で集めたデータを使って，$g_1, g_2, \dots$によるベルマン誤差を並べたベクトルを２本考えましょう．
この２つが一次従属であるとします．

このとき，$\pi_{f_1}$のデータがあれば$\pi_{f_2}$のデータは線形に復元できるので，どちらか一方のデータだけあれば良さそうです．つまり，探索の手間が一つで済んでいます．

このように，上の行列のランクが小さい場合，探索が必要な方策の数が少なくなりそうです．


---

そこでBellman rankでは，ベルマン誤差の行列に対するランクを考えます．

**Q-Bellman Rank**

仮説集合$\mathcal{H}$は，次を満たすときに$d$次元のQ-Bellman rankを持つといいます．

すべての$h \in [H]$に対して，次を満たす$W_h: \mathcal{H} \rightarrow \mathbb{R}^d$ and $X_h: \mathcal{H} \rightarrow \mathbb{R}^d$が存在する：

* $\sup _{f \in \mathcal{H}}\left\|W_h(f)\right\|_2 \leq B_W$
* $\sup _{f \in \mathcal{H}}\left\|X_h(f)\right\|_2 \leq B_X$
$$
\mathbb{E}_{a_{0: h} \sim d^{\pi_f}}\left[Q_{h, g}\left(s_h, a_h\right)-r\left(s_h, a_h\right)-V_{h+1, g}\left(s_{h+1}\right)\right]=\left\langle W_h(g)-W_h\left(f^{\star}\right), X_h(f)\right\rangle
$$

ここで，$W_h$や$X_h$を学習者が知る必要はありません．

この期待ベルマン誤差は，「（各状態行動の誤差 × その状態行動に行く確率）の線形和」になっています．
例えばテーブルMDPでベルマン誤差を調べるためには，各状態行動を調べれば十分です．このように構造化されている場合はベルマン誤差を簡単にしらべることができるので，ベルマンランクが小さくなります．

---

**V-Bellman rank**

ベルマン方程式：$V_h(s)-\max _a\left(r_h(s, a)+\mathbb{E}_{s^{\prime} \sim P_h(\cdot \mid s, a)}\left[V_{h+1}\left(s^{\prime}\right)\right]\right)=0$ より，V値に対するBellman rankを考えても良さそうです．

仮説集合$\mathcal{H}$は，次を満たすときに$d$次元のV-Bellman rankを持つといいます．

$$
\mathbb{E}_{a_{0: h-1} \sim d^{\pi_f}, a_h=\pi_g\left(s_h\right)}\left[V_{h, g}\left(s_h\right)-r\left(s_h, a_h\right)-V_{h+1, g}\left(s_{h+1}\right)\right]=\left\langle W_h(g)-W_h\left(f^{\star}\right), X_h(f)\right\rangle
$$

---

**QとVの違い**

Q-Bellman rankでは$a_h$が$\pi_f$から出てましたが，V-Bellman rankでは$a_h$は$\pi_g$から出てます．
このとき，ベルマン方程式の$\arg \max$が$\pi_g$と一致しない場合などに影響が出てきます．



**方策のクラスを別で考える場合**

上では貪欲方策だけ考え，そしてベルマン最適方程式について考えていたので，$Q$関数の仮説集合と方策の仮説集合が同じでした．
つまり，任意の$f \in \mathcal{H}$に対して，対応する方策とQ関数のペア
$$
\{(\pi_f, Q_f): f \in \mathcal{F}\}
$$
の形になっていました．
そして，「このペアがvalidである」≒「このペアがベルマン最適方程式を満たす」
関係になっています．

ただ，別に貪欲方策だけ考える必要はありません．
方策のクラス$\Pi\subset \mathcal{S} \to \mathcal{A}$を用意し，価値関数のクラスを$\mathcal{G}$として，そのペアを考えることができます．

## Low Bellman rankなMDPの例

---

**テーブルMDP**

$$
\begin{aligned}
&\mathbb{E}_{a_{0: h} \sim d^\pi f}\left[Q_{h, g}\left(s_h, a_h\right)-r\left(s_h, a_h\right)-V_{h+1, g}\left(s_{h+1}\right)\right]\\
=
&\sum_{s, a \in \mathcal{S} \times \mathcal{A}}
d_h^{\pi_f}(s, a)
\left(Q_{h, g}\left(s_h, a_h\right)-r\left(s_h, a_h\right)-V_{h+1, g}\left(s_{h+1}\right)\right)
\end{aligned}
$$

なので，$d=|S||A|$のベルマンランクになります（多分もうちょっと丁寧にやれば$d=|S|$でいける）

---

**$Q^\star$と$V^\star$の両方が線形に実現可能なとき**

これはBilinear rankの論文が初出っぽいですね．

例えば$Q^\star$と$V^\star$が線形に実現可能な場合を考えましょう．つまり，

$$
Q_h^{\star}(s, a)=\left(\theta_h^{\star}\right)^{\top} \phi(s, a) \text { and } V_h^{\star}(s)=\left(w_h^{\star}\right)^{\top} \psi(s) \text {. }
$$

が成り立っているとします．
そして，仮説集合を

$$
\mathcal{H}_h=\left\{(\theta, w):\|\theta\|_2 \leq W_1,\|w\|_2 \leq W_2, \forall s, \max _a \theta^{\top} \phi(s, a)=w^\top \psi(s)\right\}
$$

とします．$Q_{h, g}(s, a)=\theta_h^{\top} \phi(s, a), V_{h, g}(s)=w_h^{\top} \psi(s)$とすると，

$$
\begin{aligned}
& \mathbb{E}_{s, a \sim d_h^{\pi_f}}\left[\theta_h^{\top} \phi(s, a)-r(s, a)-\mathbb{E}_{s^{\prime} \sim P_h(\cdot \mid s, a)}\left[w_h^{\top} \psi\left(s^{\prime}\right)\right]\right] \\
& =\mathbb{E}_{s, a \sim d_h^{\pi_f}}\left[\theta_h^{\top} \phi(s, a)-Q_h^{\star}(s, a)+\mathbb{E}_{s^{\prime} \sim P_h(\cdot \mid s, a)}\left[V_h^{\star}\left(s^{\prime}\right)\right]-\mathbb{E}_{s^{\prime} \sim P_h(\cdot \mid s, a)}\left[w_h^{\top} \psi\left(s^{\prime}\right)\right]\right] \\
& =\mathbb{E}_{s, a \sim d_h^{\pi_f}}\left[\left(\theta_h-\theta_h^{\star}\right)^{\top} \phi(s, a)+\left(w_h-w_h^{\star}\right)^{\top} \mathbb{E}_{s^{\prime} \sim P_h(\cdot \mid s, a)}\left[\psi\left(s^{\prime}\right)\right]\right] \\
& =\left\langle\left[\begin{array}{c}
\theta_h-\theta_h^{\star} \\
w_h-w_h^{\star}
\end{array}\right], \mathbb{E}_{s, a \sim d_h^{\pi_f}}\left[\begin{array}{c}
\phi(s, a) \\
\mathbb{E}_{s^{\prime} \sim P_h(\cdot \mid s, a)}\left[\psi\left(s^{\prime}\right)\right]
\end{array}\right]\right\rangle .
\end{aligned}
$$

なので，

$$
W_h(g):=\left[\begin{array}{c}
\theta_h \\
w_h
\end{array}\right] \text {, and } X_h(f):=\left[\begin{array}{c}
\phi(s, a) \\
\mathbb{E}_{s^{\prime} \sim P_h(\cdot \mid s, a)}\left[\psi\left(s^{\prime}\right)\right]
\end{array}\right]
$$

とすれば，この仮説集合$\mathcal{H}$はQ-Bellman rankが$2d$であることがわかります．


---

**Linear Bellman completeness**

[RL_General_linear_Bellman_completeness.ipynb](RL_General_linear_Bellman_completeness.ipynb)で学んだLinear bellman completenessの場合，Bellman rankが小さくなることを見てみましょう．
（ちなみにLinear Bellman completenessはContextual Linear banditも一般化しています）．

このとき，$Q_h^{\star}(s, a)=\left(\theta_h^{\star}\right)^{\top} \phi(s, a), \forall s, a, h$であることに注意しましょう．
そこで，仮説集合として
$$
\mathcal{H}_h=\left\{\theta^{\top} \phi(s, a):\|\theta\|_2 \leq W,\left\|\theta^{\top} \phi(\cdot, \cdot)\right\|_{\infty} \leq H\right\}
$$
を考えます．
仮説について，$Q_{h, f}(s, a):=\theta_h^{\top} \phi(s, a), Q_{h, g}(s, a)=w_h^{\top} \phi(s, a)$とします．

また，$\sup _{s, a}\|\phi(s, a)\|_2 \leq 1$, and $\sup _{h, \theta \in \mathcal{H}_h}\left\|\mathcal{T}_h(\theta)\right\|_2 \leq W$も仮定します．

このとき，
$$
\begin{aligned}
& \mathbb{E}_{s_h, a_h \sim d_h^{\pi_f{ }^f}, s_{h+1} \sim P_h\left(\cdot \mid s_h, a_h\right)}\left[w_h^{\top} \phi\left(s_h, a_h\right)-r\left(s_h, a_h\right)-\max _{a^{\prime}} w_{h+1}^{\top} \phi\left(s_{h+1}, a^{\prime}\right)\right] \\
& \quad=\mathbb{E}_{s_h, a_h \sim d_h^{\pi_f}}\left[w_h^{\top} \phi\left(s_h, a_h\right) \mathcal{T}_h\left(w_{h+1}\right)^{\top} \phi\left(s_h, a_h\right)\right] \\
& \quad=\left\langle w_h-\mathcal{T}_h\left(w_{h+1}\right), \mathbb{E}_{s_h, a_h \sim d_h^{\pi_f}}\left[\phi\left(s_h, a_h\right)\right]\right\rangle .
\end{aligned}
$$
とすると，
Bellman rankの定義に当てはまることがわかります．

---


**$Q^\star$-state abstraction**

これは[A unifying framework for computational reinforcement learning theory](https://rucore.libraries.rutgers.edu/rutgers-lib/26345/)
で初めて考えられたっぽいです．

この問題では，次を満たすabstraction関数$\xi: \mathcal{S} \mapsto \mathcal{Z}$が存在することを仮定します（ここで$|\mathcal{Z}|< |\mathcal{S}|$です．）
$$
\forall s, s^{\prime}, \xi(s)=\xi\left(s^{\prime}\right) \Rightarrow Q_h^{\star}(s, a)=Q_h^{\star}\left(s^{\prime}, a\right), \forall a \in \mathcal{A}, V_h^{\star}(s)=V_h^{\star}\left(s^{\prime}\right)
$$

これは$\xi(s)$に価値関数が依存するようなPOMDPとみなすことができます（つまり，状態空間は大きいが観測空間は小さい．状態ではなく観測に対して反応する方策は[reactive agent](https://www.cs.ubc.ca/~murphyk/Papers/pomdp.pdf)って呼ばれるみたいです．）．

この設定は$Q^\star$/$V^\star$-realizableな設定の特殊ケースなので，Bellman rankも低くなります（証明は[RL Theory and Algorithms](https://rltheorybook.github.io/rltheorybook_AJKS.pdf)参照）．
ちなみに状態と行動の両方をaggregateするケースはまだ未解決です（Bilinear rankにも含まれません）．



---

**Low occupancy measure**

次の関係が成り立っているとき，そのMDPはLow-occupancy measureである，といいます．
任意の$h$について，任意の$f\in \mathcal{H}$に対して次を満たす$\beta$と$\phi$が存在する：

$$
d_h^{\pi_f}(s, a)=\left\langle\beta_h(f), \phi_h(s, a)\right\rangle
$$

このとき，
$$
\begin{aligned}
& \mathbb{E}_{s, a \sim d_h^{\pi_f}}\left[Q_{h, g}(s)-r(s, a)-\mathbb{E}_{s^{\prime} \sim P_h(\cdot \mid s, a}\left[V_{h+1, g}\left(s^{\prime}\right)\right]\right] \\
& =\beta_h(f)^{\top}\left(\sum_{s, a} \phi_h(s, a)\left[Q_{h, g}(s, a)-r(s, a)-\mathbb{E}_{s^{\prime} \sim P_h(\cdot \mid s, a)}\left[V_{h+1, g}\left(s^{\prime}\right)\right]\right]\right)
\end{aligned}
$$

なので，これはLow bellman rankです．（ちなみに状態について分解できる場合はV-bellman rankになります）．


---

**Feature selection in low-rank MDP（表現学習）**

これはLinear MDPに似てますが，特徴ベクトルは未知であることを仮定します．
次のLow rank MDPを考えましょう．
$$
P_h\left(s^{\prime} \mid s, a\right)=\mu^{\star}\left(s^{\prime}\right)^{\top} \phi^{\star}(s, a), r(s, a)=\left(\theta^{\star}\right)^{\top} \phi^{\star}(s, a)
$$

仮説集合を$\mathcal{H}_h:=\left\{w^{\top} \phi(s, a):\|w\|_2 \leq W, \phi \in \Phi\right\}$とします（Linear MDPでは$\phi$が既知なので，仮説集合は$w$だけで十分です）．
すると，
$$
\begin{aligned}
& \mathbb{E}_{s \sim d_h^{\pi_f}}\left[V_{h, g}(s)-r\left(s, \pi_{h, g}(s)\right)-\mathbb{E}_{s^{\prime} \sim P_h\left(\cdot \mid s, \pi_{h, g}(s)\right)}\left[V_{h+1, g}\left(s^{\prime}\right)\right]\right] \\
& =\mathbb{E}_{\tilde{s}, \tilde{a} \sim d_{h-1}^{\pi_f}} \mathbb{E}_{s \sim P_{h-1}(\cdot \mid \tilde{s}, \tilde{a})}\left[V_{h, g}(s)-r\left(s, \pi_{h, g}(s)\right)-\mathbb{E}_{s^{\prime} \sim P_h\left(\cdot \mid s, \pi_{h, g}(s)\right)}\left[V_{h+1, g}\left(s^{\prime}\right)\right]\right] \\
& =\mathbb{E}_{\tilde{s}, \tilde{a} \sim d_{h-1}^{\pi_f}} \int_s \mu^{\star}(s)^{\top} \phi^{\star}(\tilde{s}, \tilde{a})\left[V_{h, g}(s)-r\left(s, \pi_{h, g}(s)\right)-\mathbb{E}_{s^{\prime} \sim P_h\left(\cdot \mid s, \pi_{h, g}(s)\right)}\left[V_{h+1, g}\left(s^{\prime}\right)\right]\right] d(s) \\
& =\left\langle\int_s \mu^{\star}(s)\left[V_{h, g}(s)-r\left(s, \pi_{h, g}(s)\right)-\mathbb{E}_{s^{\prime} \sim P_h\left(\cdot \mid s, \pi_{h, g}(s)\right)}\left[V_{h+1, g}\left(s^{\prime}\right)\right]\right], \quad \mathbb{E}_{\tilde{s}, \tilde{a} \sim d^{\pi_f}}\left[\phi^{\star}(\tilde{s}, \tilde{a})\right]\right\rangle
\end{aligned}
$$
とかけるので，これは$V$-Bellman rankが$d$とみなせます．


---

**Block MDP**

Block MDPは状態空間の他に，サイズの小さいhidden space $\mathcal{Z}$が存在するモデルです．
これはLow-rankMDPの特別ケースとみなせるので，Bellman rankが小さくなります（$|\mathcal{Z}|$になります．証明は教科書参照）．

---

**sparse linear MDP**

TODO: 



## アルゴリズム

Bellman rankを使用するアルゴリズムでは行動空間は離散である必要があるっぽい？なのでLQRとかは一般化できない．